### dNBR to Geotiff (Dask)
Created by Amos Bennett.<br>
Last Updated 23 Sep 20. <br>

__Updates:__ 

Generates a dNBR geotiff (process using dask arrays). Unsure about lazy_load.

In [ ]:
import sys
import datacube
from datacube.helpers import write_geotiff
from datacube.utils import cog
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/home/554/ab4513/dea-notebooks/Scripts")
import dea_datahandling
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_plotting import display_map
from dea_bandindices import calculate_indices

In [ ]:
print(datacube.__version__)

In [ ]:
dc = datacube.Datacube(app="dNBR")

In [ ]:
# Set the central latitude and longitude
central_lat = -35.783333
central_lon = 148.016667
crs = 'EPSG:32755'

# Canberra 148.6547665°E 35.5655761°S
# Kosciuszko 148.3517111°E 36.1864717°S
# Tumbarumba 148.016667°E 35.783333°S

# 0.1° approximately equal to 11.1km distance.

# Set the buffer to load around the central coordinates (even numbers such as 0.2, 1.0, 2.2 etc) in degrees (lat, lon)
buffer = 0.6

# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

# display_map(x=study_area_lon, y=study_area_lat, margin=-0.2)

In [ ]:
# Key Dates
prefire_start = '2019-11-01'
prefire_end = '2020-01-06'
postfire_start = '2020-01-07'
postfire_end = '2020-05-01'

In [ ]:
query_1 = {"x": (central_lon - buffer, central_lon + buffer),
         "y": (central_lat - buffer, central_lat + buffer),
         "time": (prefire_start, prefire_end),
         "output_crs": "EPSG:32755",
         "resolution": (-10, 10)}

prefire_data = load_ard(dc=dc,
    products=['s2a_ard_granule', 's2b_ard_granule'],
    measurements=['nbart_nir_1', 'nbart_swir_3'],
    min_gooddata=0,
    dask_chunks={'x': 'auto', 'y': 'auto'},
    lazy_load=True,
    group_by='solar_day',**query_1)


prefire_image = prefire_data.median(dim='time')
prefire_image = calculate_indices(prefire_image,
                                  index='NBR', 
                                  collection='ga_s2_1', 
                                  drop=False)
prefire_burnratio = prefire_image.NBR
prefire_burnratio.data

In [ ]:
query_2 = {"x": (central_lon - buffer, central_lon + buffer),
         "y": (central_lat - buffer, central_lat + buffer),
         "time": (postfire_start, postfire_end),
         "output_crs": "EPSG:32755",
         "resolution": (-10, 10)}

postfire_data = load_ard(dc=dc,
    products=['s2a_ard_granule', 's2b_ard_granule'],
    measurements=['nbart_nir_1', 'nbart_swir_3'],
    min_gooddata=0,
    dask_chunks={'x': 'auto', 'y': 'auto'},
    lazy_load=True,
    group_by='solar_day',**query_2)

postfire_image = postfire_data.median(dim='time')
postfire_image = calculate_indices(postfire_image,
                                  index='NBR', 
                                  collection='ga_s2_1', 
                                  drop=False)
postfire_burnratio = postfire_image.NBR
postfire_burnratio.data

In [ ]:
%%time

# delta_NBR = prefire_NBR - postfire_NBR.compute()
delta_NBR = prefire_burnratio - postfire_burnratio
# delta_NBR.data
delta_NBR.compute()
dnbr_dataset = delta_NBR.to_dataset(name='delta_NBR')

# cog.write_cog(delta_NBR, "./dask_testing/tumbarumba.tif")
write_geotiff(f'./scratch/wj97/ab4513/dNBR_tiffs/tumbarumba_dNBR_dask.tif', dnbr_dataset)